In [27]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py

import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv")

table = ff.create_table(df)
# py.iplot(table, filename='jupyter_table')

df.std()


import plotly.plotly as py
from plotly.graph_objs import *

data = [Bar(x=df.School,
            y=df.Gap)]

py.iplot(data, filename='jupyter_basic_bar')

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~willzjc/12.embed" height="525px" width="100%"></iframe>

In [28]:
import plotly.plotly as py
from plotly.graph_objs import *

trace_women = Bar(x=df.School,
                  y=df.Women,
                  name='Women',
                  marker=dict(color='#ffcdd2'))

trace_men = Bar(x=df.School,
                y=df.Men,
                name='Men',
                marker=dict(color='#A2D5F2'))

trace_gap = Bar(x=df.School,
                y=df.Gap,
                name='Gap',
                marker=dict(color='#59606D'))

data = [trace_women, trace_men, trace_gap]
layout = Layout(title="Average Earnings for Graduates",
                xaxis=dict(title='School'),
                yaxis=dict(title='Salary (in thousands)'))
fig = Figure(data=data, layout=layout)

py.iplot(fig, filename='jupyter_styled_bar')

<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~willzjc/15.embed" height="525px" width="100%"></iframe>

In [31]:
import plotly.plotly as py
from plotly.graph_objs import *
import pandas as pd
import os
cwd=os.getcwd().replace('\\','/')
#github/willzjc.github.io
if 'historical' not in cwd:
    cwd=cwd+'/github/willzjc.github.io/python/historical_price/'
os.chdir(cwd)



In [38]:
cdf=pd.read_csv('correl.csv')

trace_clothes = Bar(x=cdf.offset,
                    y=cdf.clothes,
                    name='clothes',
                    marker=dict(color='#ffcdd2'))

trace_food = Bar(x=cdf.offset,
                 y=cdf.food,
                 name='food',
                 marker=dict(color='#A2D5F2'))

trace_furniture = Bar(x=cdf.offset,
                      y=cdf.furniture,
                      name='furniture',
                      marker=dict(color='#59606D'))

trace_total = Scatter(
    x=cdf.offset
    , y=cdf.total
    , name='total'
    , mode='lines'
    # , marker=dict(color='#C9606D')
)

data = [trace_clothes, trace_food, trace_furniture, trace_total]
layout = Layout(title="Correlation of CPI / Average prices for various sectors",
                xaxis=dict(title='offset'),
                yaxis=dict(title='Correl'))
fig = Figure(data=data, layout=layout)

py.iplot(fig, filename='sector_correlation')


<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plot.ly/~willzjc/17.embed" height="525px" width="100%"></iframe>

In [9]:
import plotly.plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
print cf.__version__

0.12.1


In [17]:
import os
import pandas as pd
import numpy as np
import datetime
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as FF
import sys

import matplotlib.pyplot as plt

from scipy.interpolate import spline

# import os
# os.chdir('/Users/zijianchen/PycharmProjects/github/webgit/willzjc.github.io/python/historical_price')

if sys.version_info[0] < 3:
    from StringIO import StringIO
else:
    from io import StringIO

def drawdf(df,selfdraw=True):


    #Normalize DF First

    df['combined']=0
    for eachitem in df.columns:
        if not 'date' in eachitem:
            df[eachitem] = (df[eachitem] - df[eachitem].mean()) / (df[eachitem].std())
            df['combined']=df['combined']+df[eachitem]

    # Get Total Rating based on average of
    df['avg']=df[[c for c in df.columns[1:]]].mean(axis=1)

    # df=df.replace('\sUSD','',regex=True).apply(pd.to_numeric, errors='ignore')
    # df['dateindex']=pd.to_datetime(df['date'])
    df['date']=pd.to_datetime(df['date'])
    df=df.set_index(pd.DatetimeIndex(df['date']))


    cpi=pd.read_csv('ref/AUCPI',delimiter='\t')
    cpi['date']=pd.to_datetime(cpi['date'])

    # Timeshift
    timeshift=31+92
    cpi['date'] = cpi['date'] - datetime.timedelta(days=(timeshift))

    cpi=cpi.set_index(pd.DatetimeIndex(cpi['date']))

    print cpi

    # Normalize CPI
    # index = pd.date_range(cpi['date'].max(),cpi['date'].min())
    # index = pd.date_range(cpi['date'])
    # values = pd.Series(cpi.values, index=index)

    # Read above link about the different Offset Aliases, S=Seconds
    # resampled_values = values.resample('2.5D')
    # cpi.diff()  # compute the difference between each point!


    cpi['CPI'] = (cpi['CPI'] - cpi['CPI'].mean()) / (cpi['CPI'].std())


    # xnew = np.linspace(df['combined'].min(), df['combined'].max(), 300)  # 300 represents number of points to make between T.min and T.max

    # power_smooth = spline(df['date'], df['combined'], xnew)

    # plt.plot(xnew, power_smooth)
    # plt.show()

    ##### Concat both tables #####
    df=pd.concat([cpi,df],axis=1, join='inner')
    # print type(df),type(cpi)

    # ##### Massaging Data #####
    # ### resample
    # df = df.resample('M').mean()
    # df = df.resample('M')
    # df = df.resample('D')
    # # ### interpolate
    # df= df.interpolate(method='cubic')

    # df.plot(kind='bar')
    # df['bad_rate'].plot(secondary_y=True)

    #### PLOTTING SECTION #####
    # cpi=cpi.interpolate(method='cubic')
    # tsint.plot()
    # tsint.plot(y='combined', use_index=True)
    # tsint.plot()
    # ax = cpi.plot(y='CPI', use_index=True, kind='bar')
    # tsint.plot.line(ax=ax,y='avg',use_index=True,secondary_y=True,kind='line')
    # cpi_s.plot(ax=ax)
    # result.plot(y='avg',kind='bar')
    # result.plot(y='CPI',secondary_y=True)
    # result.plot(y=['avg','CPI'])
    f, axarr = plt.subplots(2)


    df['positive']=df['avg'] > 0
    
    if selfdraw:
        df.CPI.plot(legend=True, ax=axarr[0])
        df.avg.plot(ax=axarr[1], kind='bar', legend=True,color=df.positive.map({True: 'g', False: 'r'}))
        plt.show()
    
    return df

def readfiles():

    bol_recursive=False
    df=pd.DataFrame(columns=['date'])

    basepath=os.getcwd().replace('\\','/') + '/' + 'ref' + '/'
    print basepath

    files=[]

    for str_dirname, lst_subdirs, lst_files in os.walk(basepath):
        if not bol_recursive:
            while len(lst_subdirs) > 0:
                lst_subdirs.pop()
                for file in lst_files:
                    if '.csv' in file:
                        with open(basepath+file,'rb') as f:
                            files.append(f.read())
                            f.close()

    for f in files:

        headers = {}
        buffer=[]
        headermode = True
        for line in f.split('\n'):
            if headermode and 'Date,' in line:
                headermode=False
            elements=line.strip().split(',')
            if len(elements) < 3 and len(elements) > 1:         # filter out header info
                headers[elements[0].strip()]=elements[1].strip()
            else:
                if len(elements)>1 and not headermode:
                    buffer.append(line)

        # Read file stream CSV
        currentdf = pd.read_csv(StringIO('\n'.join(buffer)))

        # Replace Strings
        currentdf = currentdf.replace('\sUSD', '', regex=True).apply(pd.to_numeric, errors='ignore')
        try:
            df['date'] = currentdf['Date']
        except:
            # TODO
            print 'Deal with this exception another time'

        df[headers['Keywords']] = currentdf['Average Selling Price']

    # df['combined']=0

    return df

def main():
    df=readfiles()
    drawdf(df)

if __name__ == "__main__":
    main()

/Users/zijianchen/PycharmProjects/github/webgit/willzjc.github.io/python/historical_price/ref/
                 date    CPI
date                        
2014-10-29 2014-10-29  107.5
2015-01-28 2015-01-28  108.0
2015-04-30 2015-04-30  108.4
2015-07-30 2015-07-30  107.6
2015-10-30 2015-10-30  108.2
2016-01-29 2016-01-29  108.6
2016-04-30 2016-04-30  109.4
2016-07-31 2016-07-31  110.0
2016-10-30 2016-10-30  110.5
2017-01-29 2017-01-29  110.7
2017-05-01 2017-05-01  111.4
2017-07-31 2017-07-31  112.1


In [7]:
from IPython.display import clear_output
clear_output(wait=True)


data="""Seroes	Index Numbers ;  Food and non-alcoholic beverages ;  Australia ;	Index Numbers ;  Alcohol and tobacco ;  Australia ;	Index Numbers ;  Clothing and footwear ;  Australia ;	Index Numbers ;  Housing ;  Australia ;	Index Numbers ;  Furnishings, household equipment and services ;  Australia ;	Index Numbers ;  Health ;  Australia ;	Index Numbers ;  Transport ;  Australia ;	Index Numbers ;  Communication ;  Australia ;	Index Numbers ;  Recreation and culture ;  Australia ;	Index Numbers ;  Education ;  Australia ;	Index Numbers ;  Insurance and financial services ;  Australia ;	Index Numbers ;  All groups CPI ;  Australia ;
Unit	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers	Index Numbers
Series Type	Original	Original	Original	Original	Original	Original	Original	Original	Original	Original	Original	Original
Data Type	INDEX	INDEX	INDEX	INDEX	INDEX	INDEX	INDEX	INDEX	INDEX	INDEX	INDEX	INDEX
Frequency	Quarter	Quarter	Quarter	Quarter	Quarter	Quarter	Quarter	Quarter	Quarter	Quarter	Quarter	Quarter
Collection Month	3	3	3	3	3	3	3	3	3	3	3	3
Series Start	Sep-1972	Sep-1972	Sep-1972	Sep-1972	Sep-1972	Sep-1989	Sep-1972	Sep-1972	Sep-1989	Mar-1982	Jun-2005	Sep-1948
Series End	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017	Dec-2017
No. Obs	182	182	182	182	182	114	182	182	114	144	51	278
Series ID	A2325891R	A2326116V	A2325936J	A2325981V	A2326026R	A2331111C	A2326071A	A2331201J	A2331246L	A2331426W	A2332596F	A2325846C
Jun-2005	78.9	72.7	101.0	73.6	94.5	75.8	85.6	97.4	96.0	71.8	85.1	82.6
Sep-2005	79.5	73.4	101.1	74.6	94.8	75.0	88.4	96.7	97.2	71.9	85.2	83.4
Dec-2005	81.0	73.8	101.0	75.1	95.4	74.2	88.0	96.0	97.2	71.9	86.5	83.8
Mar-2006	82.1	74.7	98.4	75.6	95.1	77.5	89.1	96.4	97.8	76.0	86.0	84.5
Jun-2006	85.5	75.4	99.3	76.2	95.8	79.3	92.1	96.5	97.5	76.0	87.0	85.9
Sep-2006	87.5	75.8	99.4	77.1	97.1	78.7	92.5	97.1	98.2	75.3	87.3	86.7
Dec-2006	87.9	76.4	98.9	77.5	97.2	78.1	89.0	97.5	99.5	75.4	87.7	86.6
Mar-2007	85.8	77.0	98.6	78.3	96.4	80.9	89.5	97.7	98.8	79.1	87.5	86.6
Jun-2007	87.3	77.7	100.0	78.9	97.8	82.6	92.3	97.9	98.4	79.2	88.3	87.7
Sep-2007	89.0	78.1	100.3	80.4	95.5	82.1	91.7	97.9	99.7	78.5	90.1	88.3
Dec-2007	88.9	79.1	100.5	81.3	96.3	81.3	93.9	97.9	100.6	78.5	92.0	89.1
Mar-2008	90.8	80.0	98.1	82.8	95.6	84.5	95.7	97.9	100.2	82.5	93.5	90.3
Jun-2008	90.7	81.4	101.0	83.7	97.2	86.6	98.6	97.9	100.0	82.6	97.0	91.6
Sep-2008	92.0	82.6	100.3	85.8	96.4	86.4	99.6	98.1	101.3	82.2	98.7	92.7
Dec-2008	93.9	83.7	100.7	86.5	96.7	85.3	92.7	98.5	101.9	82.2	98.4	92.4
Mar-2009	96.0	84.5	100.2	87.3	97.5	89.0	91.3	98.9	100.7	86.7	92.2	92.5
Jun-2009	95.1	85.3	102.3	88.0	99.6	91.0	92.8	99.1	100.7	86.7	90.7	92.9
Sep-2009	94.3	85.9	102.6	90.6	100.2	90.1	94.6	99.1	101.4	86.8	91.5	93.8
Dec-2009	95.7	86.4	102.8	91.3	100.2	89.3	93.8	99.1	102.9	86.8	92.2	94.3
Mar-2010	96.7	87.5	98.4	92.6	98.9	93.5	95.0	99.0	101.9	91.6	94.1	95.2
Jun-2010	96.4	92.7	98.4	93.2	99.8	95.6	95.7	98.9	100.1	91.7	94.2	95.8
Sep-2010	95.9	95.5	99.7	95.2	100.6	94.9	95.1	98.7	100.8	91.8	94.7	96.5
Dec-2010	98.0	96.2	97.9	95.8	100.0	93.7	95.4	98.7	101.0	91.8	94.3	96.9
Mar-2011	100.9	97.3	97.2	97.1	98.4	97.4	97.9	98.8	100.4	97.0	96.7	98.3
Jun-2011	102.3	97.9	99.5	97.5	99.8	99.4	99.1	99.3	99.8	97.1	98.2	99.2
Sep-2011	102.0	98.4	101.0	99.2	100.1	98.4	99.2	99.2	100.7	97.1	98.9	99.8
Dec-2011	100.5	99.3	100.4	99.7	100.2	97.1	99.2	100.3	101.5	97.1	99.6	99.8
Mar-2012	98.4	100.7	98.6	100.3	99.1	101.5	100.3	100.4	99.5	102.9	100.5	99.9
Jun-2012	99.0	101.7	100.0	100.7	100.6	103.0	101.3	100.1	98.3	102.9	101.0	100.4
Sep-2012	100.9	102.6	100.2	103.9	101.6	105.5	100.5	100.6	99.2	103.0	101.2	101.8
Dec-2012	100.8	102.8	101.0	104.1	101.0	104.6	101.2	101.9	99.8	103.0	102.7	102.0
Mar-2013	100.0	104.4	97.1	105.4	99.7	107.7	101.7	101.9	99.0	108.9	103.4	102.4
Jun-2013	100.1	105.8	99.7	106.0	100.7	109.8	100.8	102.3	98.2	108.8	103.7	102.8
Sep-2013	100.3	106.7	100.8	108.1	101.7	109.8	103.2	102.4	100.1	108.8	104.1	104.0
Dec-2013	101.9	108.4	99.7	108.6	102.1	109.2	103.1	103.6	102.2	108.8	104.3	104.8
Mar-2014	102.2	111.5	97.6	109.2	100.6	112.0	104.2	103.8	101.7	114.4	104.7	105.4
Jun-2014	102.6	113.3	99.1	110.1	101.7	115.2	103.5	102.0	101.4	114.4	104.7	105.9
Sep-2014	103.8	114.5	98.1	110.6	102.1	115.0	103.4	100.6	102.0	114.5	105.3	106.4
Dec-2014	103.9	116.4	98.2	111.2	102.6	114.0	101.1	100.5	103.0	114.5	106.4	106.6
Mar-2015	104.1	117.3	96.9	112.1	102.1	116.9	97.7	99.1	103.7	120.6	106.6	106.8
Jun-2015	103.9	118.7	98.2	112.9	103.1	120.1	101.0	98.5	102.3	120.6	106.9	107.5
Sep-2015	104.0	120.2	97.1	113.6	103.9	120.5	101.1	96.5	103.1	120.8	107.4	108.0
Dec-2015	104.3	123.4	98.7	113.7	104.5	120.0	99.7	94.2	104.8	120.8	108.3	108.4
Mar-2016	104.1	124.5	96.1	114.0	104.1	122.3	97.2	92.8	103.8	124.6	109.0	108.2
Jun-2016	103.8	125.7	98.0	114.4	104.7	125.5	98.2	91.4	103.1	124.6	109.5	108.6
Sep-2016	105.6	127.1	98.3	115.6	105.9	125.2	97.7	89.3	103.7	124.8	110.5	109.4
Dec-2016	106.2	130.7	97.8	115.9	105.1	124.4	99.4	88.6	104.3	124.8	111.2	110.0
Mar-2017	106.0	132.1	96.4	116.8	104.0	126.9	100.9	88.3	103.6	128.7	111.9	110.5
Jun-2017	105.8	133.1	96.1	117.2	104.7	130.3	100.3	87.9	103.0	128.7	111.8	110.7
Sep-2017	104.9	136.0	95.2	119.4	105.1	130.1	100.3	86.7	104.3	128.7	112.5	111.4
Dec-2017	106.0	140.3	94.9	119.8	104.3	129.4	102.7	85.6	104.9	128.8	112.7	112.1
"""

import pandas as pd
import StringIO
df=pd.read_csv(StringIO.StringIO(data),delimiter='\t')
print df


              Seroes  \
0               Unit   
1        Series Type   
2          Data Type   
3          Frequency   
4   Collection Month   
5       Series Start   
6         Series End   
7            No. Obs   
8          Series ID   
9           Jun-2005   
10          Sep-2005   
11          Dec-2005   
12          Mar-2006   
13          Jun-2006   
14          Sep-2006   
15          Dec-2006   
16          Mar-2007   
17          Jun-2007   
18          Sep-2007   
19          Dec-2007   
20          Mar-2008   
21          Jun-2008   
22          Sep-2008   
23          Dec-2008   
24          Mar-2009   
25          Jun-2009   
26          Sep-2009   
27          Dec-2009   
28          Mar-2010   
29          Jun-2010   
30          Sep-2010   
31          Dec-2010   
32          Mar-2011   
33          Jun-2011   
34          Sep-2011   
35          Dec-2011   
36          Mar-2012   
37          Jun-2012   
38          Sep-2012   
39          Dec-2012   
40          Mar-

In [6]:
from IPython.display import clear_output
clear_output(wait=True)

df=readfiles()
df=drawdf(df,selfdraw=False)

df = cf.datagen.lines()

py.iplot([{
    'x': df.index,
    'y': df[col],
    'name': col
}  for col in df.columns], filename='cufflinks/simple-line')


NameError: name 'readfiles' is not defined